In [ ]:

from scipy.special import gamma, polygamma

import sympy
from sympy.stats import Beta, density, E, variance
from sympy import Symbol, simplify, pprint, expand_func, S, pi, Matrix, Array, symbols, IndexedBase, Idx, lambdify
from sympy import Sum, Integral, IndexedBase, Function
from sympy.stats import Expectation


In [ ]:
importlib.reload(netx)
importlib.reload(dl)

model = netx.NaiveCNN('mse').cuda()
par_model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])

loss_fn = nn.modules.loss.MSELoss()
#loss_fn = nn.modules.loss.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=.0005)
epochs = 10
for epoch in range(epochs):
    for batch_idx, (imgs, huy) in enumerate(train_loader):
        imgs, huy = imgs.cuda().float(), huy.cuda().float()
        loss = loss_fn(par_model(imgs, huy[:,1]), torch.round(huy[:,-1]*6))
        #loss = loss_fn(par_model(imgs, huy[:,1].float()), torch.round(huy[:,-1]*6).long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * M, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

print_stats(test=False)
print_stats(test=True)

In [ ]:
y = torch.tensor(y_train, dtype=dtype).cuda()
u = torch.tensor(u_train, dtype=dtype).cuda()
model = MixModel(K).cuda()
par_model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])

clipper = ProbClipper()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
yu = torch.stack([y,u], 1)

In [ ]:
### logit-normal dist
sigma = Symbol("sigma", positive=True)
mu = Symbol("mu", positive=True)
y = Symbol("y", positive=True)
pdf = sympy.exp(-(sympy.log(y/(1-y)) - mu)**2/(2*sigma**2)) * (1/(y*(1-y))) / (sigma*(2*pi)**.5)

dP_dy = pdf.diff(y).subs({sigma:1, mu:0})

scipy.optimize.broyden2(lambdify(y, dP_dy), [0.3])

In [ ]:
### beta distribution
a_y = Symbol("a", positive=True)
b_y = Symbol("b", positive=True)
y = Symbol("y", positive=True)
Y = sympy.stats.Beta('Y', a_y, b_y)
pdf = density(Y)(y)

pdf.diff(b_y)

dP_db = pdf.diff(b_y).subs({a_y:1, y:.5})

dP_da = pdf.diff(alpha).subs({beta:1, y:.5})

scipy.optimize.broyden1(lambdify(b_y, dP_db), [1])

np.argmax(stats.beta.pdf(.5, alphas, betas))

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [ ]:
def plot_beta_2d_pdf(a1,b1,a2,b2):
    cuda_to_np = lambda x: x.detach().cpu().numpy()[0]
    a1 = cuda_to_np(a1)
    a2 = cuda_to_np(a2)
    b1 = cuda_to_np(b1)
    b2 = cuda_to_np(b2)
    
    mu_X = stats.beta.mean(a1,b1)
    mu_Y = stats.beta.mean(a2,b2)
    var_X = stats.beta.var(a1,b1)
    var_Y = stats.beta.var(a2,b2)
    X = [np.linspace(mu_X[i]-var_X[i]**.5, mu_X[i]+var_X[i]**.5,1000) for i in range(K)]
    Y = [np.concatenate([((1 - (X[i] - mu_X[i])**2/var_X[i]) * var_Y[i])**.5 + mu_Y[i], 
          -((1 - (X[i] - mu_X[i])**2/var_X[i]) * var_Y[i])**.5 + mu_Y[i]]) for i in range(K)]
    X = [np.tile(X[i],2) for i in range(K)]
    print(X[0].shape, Y[0].shape)
    [plt.scatter(X[i],Y[i], marker='.') for i in range(K)];
    #[plt.scatter(mu_X[i],mu_Y[i], marker='x') for i in range(K)];

# EM

In [ ]:
K = 50

a_y = IndexedBase("a_y", K, positive=True)
b_y = IndexedBase("b_y", K, positive=True)
a_u = IndexedBase("a_u", K, positive=True)
b_u = IndexedBase("b_u", K, positive=True)
P_z = IndexedBase("P_z", K, positive=True)
y = Symbol("y", positive=True)
u = Symbol("u", positive=True)
i = Idx("i", K)
k_sym = Idx("k", K)

f_YU = density(sympy.stats.Beta('fY', sympy.exp(a_y[i]), sympy.exp(b_y[i])))(y) * density(
                sympy.stats.Beta('fU', sympy.exp(a_u[i]), sympy.exp(b_u[i])))(u)
f_YU_k = density(sympy.stats.Beta('fY_k', sympy.exp(a_y[k_sym]), sympy.exp(b_y[k_sym])))(y) * density(
                sympy.stats.Beta('fU_k', sympy.exp(a_u[k_sym]), sympy.exp(b_u[k_sym])))(u)

dP_da_k = P_z[k_sym]*f_YU_k.diff(a_y[k_sym])/Sum(P_z[i]*f_YU, (i,0,K-1))
dP_da_k_py = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), dP_da_k)

dP_db_k = P_z[k_sym]*f_YU_k.diff(b_y[k_sym])/Sum(P_z[i]*f_YU, (i,0,K-1))
dP_db_k_py = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), dP_db_k)

dP_dpz_k = f_YU_k/Sum(P_z[i]*f_YU, (i,0,K-1))
dP_dpz_k_py = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), dP_dpz_k)

log_ll = sympy.log(Sum(P_z[i]*f_YU, (i,0,K-1)))
log_ll_py = lambdify((a_y, b_y, a_u, b_u, P_z, y), log_ll)

In [ ]:
def processInput(k, bounds = (.05,50)):
    dP_dpz_k_sum = lambda pz: np.sum(dP_dpz_k_py(alphas, betas, p_z[:k] + [pz] + p_z[k:], y_train, k)) - N
    dP_da_k_sum = lambda a: np.sum(dP_da_k_py(alphas[:k] + [a] + alphas[k:], betas, p_z, y_train, k))
    dP_db_k_sum = lambda b: np.sum(dP_db_k_py(alphas, betas[:k] + [b] + betas[k:], p_z, y_train, k))
    dP_dpz_k_sq = lambda pz: dP_dpz_k_sum(pz)**2
    dP_da_k_sq = lambda a: dP_da_k_sum(a)**2
    dP_db_k_sq = lambda b: dP_db_k_sum(b)**2

    try:                         
        a_est = np.exp(scipy.optimize.brentq(dP_da_k_sum))#, a=bounds[0], b=bounds[1], disp=False)
    except ValueError:
        a_est = np.exp(scipy.optimize.minimize_scalar(dP_da_k_sq).x)#, bounds=bounds, method='Bounded').x
    #alphas[k] = scipy.optimize.broyden1(dP_da_k_sum, [alphas[k]])[0]
    print(".",end="")

    try:
        b_est = np.exp(scipy.optimize.brentq(dP_db_k_sum))#, a=bounds[0], b=bounds[1], disp=False)
    except ValueError:
        b_est = np.exp(scipy.optimize.minimize_scalar(dP_db_k_sq).x)#, bounds=bounds, method='Bounded').x
    print(".",end="")

    try:
        p_z_est = scipy.optimize.brentq(dP_dpz_k_sum, a=0, b=1, disp=False)
    except ValueError:
        p_z_est = scipy.optimize.minimize_scalar(dP_dpz_k_sq, bounds=(0, 1), method='Bounded').x
    print(".",end="")
    
    return a_est, b_est, p_z_est

In [ ]:
#initialization
alphas = list(np.linspace(.1,6,K))
betas = list(np.linspace(6,.1,K))
p_z = list(np.ones(K)/K) #mixing coefficients

patience = 3
tol = .1
ll_hist = [-10000]*patience

a_est = np.zeros(K)
b_est = np.zeros(K)
p_z_est = np.zeros(K)

for _ in range(iters):
    lll = np.mean(log_ll_py(alphas, betas, p_z, y_train))
    print("Mean Log Likelihood: %.2f" % lll)
    if np.max(lll - ll_hist[-patience:]) < tol:
        break
    
    K_data = Parallel(n_jobs=num_cores)(delayed(processInput)(k) for k in range(K))
    alphas, betas, p_z_est = zip(*K_data)
    alphas = list(alphas)
    betas = list(betas)
    p_z = list(p_z_est/sum(p_z_est))
    ll_hist = ll_hist + [lll]
    
ll_hist = ll_hist[patience:]

In [ ]:
dll_da_k = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), log_ll.diff(a_y[k_sym]), modules='sympy')
dll_db_k = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), log_ll.diff(b_y[k_sym]), modules='sympy')
dll_dpz_k = lambdify((a_y, b_y, a_u, b_u, P_z, y, k_sym), log_ll.diff(P_z[k_sym]), modules='sympy')

In [ ]:
def processInput(k):
    return dll_da_k(alphas, betas, p_z, y_train, k), dll_db_k(alphas, betas, p_z, y_train, k), dll_dpz_k(alphas, betas, p_z, y_train, k)